In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re
import os

# Téléchargement de la base Sentiment140

In [2]:
import requests
import tempfile
import zipfile

url, destname = 'http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip', 'sentiment140'
temporary_location = "temp"

def download_unzip(url, dirname = tempfile.gettempdir(), destname = "file"):
    myfile = requests.get(url)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    open(os.path.join(dirname, destname + '.zip'), 'wb').write(myfile.content)
    with zipfile.ZipFile(os.path.join(dirname, destname + '.zip'), 'r') as zip_ref:
        zip_ref.extractall(os.path.join(dirname, destname))
        
download_unzip(url, dirname=temporary_location, destname=destname)

In [3]:
trainfile = os.path.join(temporary_location, destname, "training.1600000.processed.noemoticon.csv")
testfile = os.path.join(temporary_location, destname, "testdata.manual.2009.06.14.csv")
columns = ['sentiment','id','date','query_string','user','text']

In [24]:
# Import train data set
df = pd.read_csv(trainfile,
                 header=None, 
                 names=columns, 
                 encoding='latin-1')
# df = df.sample(100000)
df

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


# Préprocessing du texte

accents, contractions, lowercase, newlines, lemmatization, stemming

In [25]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [26]:
def text_preprocessing(serie):
    
    # lowercase
    serie = serie.map(lambda x: x.lower())   

    # remove extra newlines
    serie = serie.map(lambda x: re.sub(r'[\r|\n|\r\n]+', ' ', x))

    # remove @tag
    serie = serie.map(lambda x: re.sub(r'@[A-Za-z0-9]+','', x))

    # remove URL
    serie = serie.map(lambda x: re.sub('https?://[A-Za-z0-9./]+','', x))

    # remove hashtag and numbers
    serie = serie.map(lambda x: re.sub("[^a-zA-Z]", " ", x))

    # tokenization
    serie = serie.map(word_tokenize)

    # # remove stop words
    # stop_words = set(stopwords.words('english'))
    # serie = serie.map(lambda x: [word for word in x if word not in stop_words])

    # lemmatization    
    serie = serie.map(nltk.tag.pos_tag)
    serie = serie.map(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

    wordnet_lemmatizer = WordNetLemmatizer()
    serie = serie.map(lambda x: [wordnet_lemmatizer.lemmatize(word, tag) for (word, tag) in x])


    serie = serie.map(lambda x: ' '.join(word for word in x))

    return serie

# Préprocessing Sentiment140

In [ ]:
%%time
df.text = text_preprocessing(df.text)

In [ ]:
%%time
df.to_pickle(os.path.join("data", "sentiment140", "train.bz2"))